Change the root path to the current file

In [1]:
%cd /content/drive/MyDrive/PalmPrint

/content/drive/MyDrive/PalmPrint


Install relative package

In [3]:
!pip install pytorch-metric-learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 KB 10.3 MB/s eta 0:00:00


In [4]:
import os
import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet50
from pytorch_metric_learning import losses
from sklearn.model_selection import train_test_split

In [10]:
root_path = "/content/drive/MyDrive/PalmPrint/iPhone"
all_images = []
all_labels = []

label_mapping = {}
label_counter = 0

# Iterate through the root folder and all subfolders under the iPhone directory
for root, dirs, files in os.walk(root_path):
    # Get all images with common extensions in the current folder
    image_files = glob.glob(os.path.join(root, "*.JPG"))

    # Open images and append them to the all_images list, and append their labels to the all_labels list
    for img_file in image_files:
        img = Image.open(img_file)
        all_images.append(img)
        
        # Get the folder name
        folder_name = os.path.basename(os.path.dirname(img_file))
        
        # Assign a numeric label to the folder if it's not assigned yet
        if folder_name not in label_mapping:
            label_mapping[folder_name] = label_counter
            label_counter += 1

        # Get the numeric label from the label_mapping dictionary
        label = label_mapping[folder_name]
        all_labels.append(label)

print(f"Total images found: {len(all_images)}")
print(f"Total labels found: {len(all_labels)}")
print(f"Label mapping: {label_mapping}")


Total images found: 3930
Total labels found: 3930
Label mapping: {'L_016': 0, 'L_008': 1, 'L_020': 2, 'L_037': 3, 'L_024': 4, 'L_010': 5, 'L_034': 6, 'L_001': 7, 'L_029': 8, 'L_033': 9, 'L_038': 10, 'L_014': 11, 'L_036': 12, 'L_035': 13, 'L_025': 14, 'L_012': 15, 'L_023': 16, 'L_018': 17, 'L_017': 18, 'L_021': 19, 'L_030': 20, 'L_028': 21, 'L_004': 22, 'L_002': 23, 'L_027': 24, 'L_032': 25, 'L_009': 26, 'L_019': 27, 'L_015': 28, 'L_013': 29, 'L_006': 30, 'L_005': 31, 'L_003': 32, 'L_007': 33, 'L_031': 34, 'L_026': 35, 'L_022': 36, 'L_011': 37, 'L_044': 38, 'R_007': 39, 'L_039': 40, 'R_001': 41, 'R_024': 42, 'L_070': 43, 'L_064': 44, 'R_021': 45, 'R_039': 46, 'R_023': 47, 'R_029': 48, 'R_027': 49, 'L_065': 50, 'L_052': 51, 'L_067': 52, 'L_077': 53, 'L_054': 54, 'R_032': 55, 'R_009': 56, 'R_008': 57, 'L_059': 58, 'L_040': 59, 'R_050': 60, 'R_037': 61, 'L_084': 62, 'R_004': 63, 'R_013': 64, 'R_033': 65, 'R_035': 66, 'R_047': 67, 'L_057': 68, 'R_022': 69, 'L_087': 70, 'L_058': 71, 'L_050':

In [11]:
# The rest of the imports remain the same
train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)


In [12]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert the image to grayscale
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat grayscale channel to create a 3-channel image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Create a custom dataset with all_images and all_labels
train_dataset = CustomDataset(train_images, train_labels, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

test_dataset = CustomDataset(test_images, test_labels, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)



In [13]:
class model(torch.nn.Module):
    def __init__(self, out_feature=128):
        super(model, self).__init__()
        # encoder
        self.f = resnet50(pretrained=True)
        # classifier
        self.fc = nn.Linear(1000, out_feature, bias=True)

        for param in self.f.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.fc(feature)
        return out

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = model()
#encoder = nn.Sequential(*list(encoder.children())[:-1])
encoder.to(device)

num_classes = len(set(all_labels))
arcface_loss = losses.ArcFaceLoss(num_classes, 128, margin=0.5, scale=64)
arcface_loss.to(device)

optimizer = optim.Adam(encoder.parameters(), lr=0.001)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:

# Train the encoder
num_epochs = 60
loss_list = []
for epoch in range(num_epochs):
    encoder.train()
    total_loss = 0

    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        embeddings = encoder(images)
        embeddings = embeddings.view(embeddings.size(0), -1)

        loss = arcface_loss(embeddings, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch%10 == 0:
        # Save the model's state_dict
        torch.save(encoder.state_dict(), f'model_weights{epoch}.pth')

    loss_list.append(total_loss/len(train_dataloader))
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_dataloader)}")

Epoch [1/60], Loss: 0.2058198457211256
Epoch [2/60], Loss: 0.328482116907835


KeyboardInterrupt: ignored

In [16]:
def calculate_mean_feature_vectors(encoder, dataloader, num_classes, feature_dim):
    class_sum = torch.zeros(num_classes, feature_dim).cuda()
    class_count = torch.zeros(num_classes).cuda()
    # Set the model to evaluation mode
    encoder.eval()

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()

            # Encode the inputs to obtain the feature vectors
            feature_vectors = encoder(inputs)

            # Sum the feature vectors for each class
            for i in range(num_classes):
                mask = (labels == i)
                class_sum[i] += feature_vectors[mask].sum(dim=0)
                class_count[i] += mask.sum()

    # Calculate the mean feature vector for each class
    mean_feature_vectors = class_sum / class_count.view(-1, 1)

    return mean_feature_vectors

mean_feature = calculate_mean_feature_vectors(encoder, train_dataloader, num_classes, 128)
mean_feature_normalized = torch.nn.functional.normalize(mean_feature, dim=1)



In [26]:
def eval_loop(encoder, dataloader):

    encoder.eval()
    correct = 0
    total = 0


    with torch.no_grad():
        for images, labels in dataloader:
          images = images.to(device)
          labels = labels.to(device)

          embeddings = encoder(images)
          embeddings = embeddings.view(embeddings.size(0), -1)
          embeddings_normalized = torch.nn.functional.normalize(embeddings, dim=1)

          cosine_similarity = torch.matmul(embeddings_normalized, mean_feature_normalized.t())
          prediction = torch.argmax(cosine_similarity, dim=1)


          total += labels.size(0)
          correct += (prediction == labels).sum().item()

    return 100 * correct / total
for i in range(10, 70, 10):
  encoder.load_state_dict(torch.load(f'model_weights{i}.pth'))
  accuracy = eval_loop(encoder, test_dataloader)
  print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 60.94%
Test Accuracy: 67.81%
Test Accuracy: 68.70%
Test Accuracy: 70.48%
Test Accuracy: 70.87%
Test Accuracy: 70.99%


In [27]:
encoder.load_state_dict(torch.load('model_weights60.pth'))

encoder.eval()
correct = 0
total = 0

true_labels_all = []
predicted_labels_all = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        embeddings = encoder(images)
        embeddings = embeddings.view(embeddings.size(0), -1)
        embeddings_normalized = torch.nn.functional.normalize(embeddings, dim=1)

        cosine_similarity = torch.matmul(embeddings_normalized, mean_feature_normalized.t())
        prediction = torch.argmax(cosine_similarity, dim=1)

        # Store the true labels and predicted labels
        true_labels_all.extend(labels.cpu().numpy())
        predicted_labels_all.extend(prediction.cpu().numpy())

        total += labels.size(0)
        correct += (prediction == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 70.99%


In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def plot_confusion_matrix(conf_mat, labels, save_path='confusion_matrix.png'):
    plt.figure(figsize=(10, 10))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig(save_path)


conf_mat = confusion_matrix(true_labels_all, predicted_labels_all)
class_report = classification_report(true_labels_all, predicted_labels_all, output_dict=True)

# Assuming your classes are integers from 0 to num_classes - 1
labels = list(range(num_classes))



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
import pandas as pd
# Create a DataFrame from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, index=labels, columns=labels)

# Save the confusion matrix to a CSV file
conf_mat_df.to_csv('confusion_matrix.csv', index_label='True\Predicted')

# Create a DataFrame from the classification report
class_report_df = pd.DataFrame(class_report).transpose()

# Save the classification report to a CSV file
class_report_df.to_csv('classification_report.csv', index_label='Class')


In [30]:
# Create a DataFrame from the loss list
loss_list_df = pd.DataFrame(loss_list, columns=['Loss'])

# Save the loss list to a CSV file
loss_list_df.to_csv('loss_list.csv', index_label='Iteration')
